In [42]:
!pip install yt_dlp

In [43]:
import pandas as pd

video_parse_list = pd.read_csv('./videos_parse_list.csv')
video_parse_list.head()

,path,url,noplaylist
0,뱀서라이크,https://www.youtube.com/playlist?list=PLO-mt5I...,False
1,비행기슈팅,https://www.youtube.com/playlist?list=PLO-mt5I...,False
2,탑다운RPG,https://www.youtube.com/playlist?list=PLO-mt5I...,False
3,플랫포머,https://www.youtube.com/playlist?list=PLO-mt5I...,False
4,유니티6,https://www.youtube.com/watch?v=350EWYC_1ZA&li...,True


In [44]:
for row in video_parse_list.itertuples(index=False):
    path = row.path
    url = row.url
    noplaylist = row.noplaylist
    print(f"Path: {path}, URL: {url}, noplaylist: {noplaylist}")

Path: 뱀서라이크, URL: https://www.youtube.com/playlist?list=PLO-mt5Iu5TeZF8xMHqtT_DhAPKmjF6i3x, noplaylist: False
Path: 비행기슈팅, URL: https://www.youtube.com/playlist?list=PLO-mt5Iu5TeYtWvM9eN-xnwRbyUAMWd3b, noplaylist: False
Path: 탑다운RPG, URL: https://www.youtube.com/playlist?list=PLO-mt5Iu5TeYfyXsi6kzHK8kfjPvadC5u, noplaylist: False
Path: 플랫포머, URL: https://www.youtube.com/playlist?list=PLO-mt5Iu5TeZGR_y6mHmTWyo0RyGgO0N_, noplaylist: False
Path: 유니티6, URL: https://www.youtube.com/watch?v=350EWYC_1ZA&list=PLO-mt5Iu5TeZWoYS0KENkohpFgjMnUBgk&index=1, noplaylist: True
Path: 유니티6, URL: https://www.youtube.com/watch?v=Q7mHfbEzaJM&list=PLO-mt5Iu5TeZWoYS0KENkohpFgjMnUBgk&index=2, noplaylist: True
Path: 유니티6, URL: https://www.youtube.com/watch?v=r4aZypBrFHE, noplaylist: True
Path: 유니티6, URL: https://www.youtube.com/watch?v=3mSO1sMdGxM, noplaylist: True


In [45]:
import yt_dlp
import os

def get_video_or_playlist_info(url, noplaylist=True):
    """
    입력된 URL이 재생목록인지 단일 동영상인지 확인하고 정보를 반환합니다.
    :param url: 유튜브 URL
    :param noplaylist: True일 경우 재생목록을 단일 동영상으로 처리
    :return: 동영상 또는 재생목록의 정보
    """
    ydl_opts = {
        'quiet': True,
        'extract_flat': not noplaylist,  # 재생목록일 경우 정보만 가져옴
        'noplaylist': noplaylist,  # 사용자의 선택에 따라 재생목록 전체 또는 단일 동영상 처리
        'geo_bypass': True,  # 지역 제한 우회
        'geo_bypass_country': 'KR',  # 한국 지역으로 설정
        'http_headers': {
            'Accept-Language': 'ko'  # HTTP 요청에 한국어 설정
        }
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return info


def download_audio(youtube_url: str, output_path: str = "./", noplaylist=True) -> None:
    """
    유튜브 동영상에서 오디오만 다운로드합니다.
    :param youtube_url: 유튜브 동영상 또는 재생목록 URL
    :param output_path: 오디오 파일이 저장될 경로
    :param noplaylist: True일 경우 재생목록을 단일 동영상으로 처리
    """
    # 디렉토리가 없으면 생성
    os.makedirs(output_path, exist_ok=True)

    # URL 정보 가져오기
    info = get_video_or_playlist_info(youtube_url, noplaylist=noplaylist)

    # 단일 동영상 처리
    if 'entries' not in info:
        title = info.get('title', 'Unknown Title')  # 타이틀 자동 추출
        file_path = os.path.join(output_path, f"{title}.mp3")

        # 파일이 존재하는지 확인
        if os.path.exists(file_path):
            print(f"파일이 이미 존재합니다: {file_path}")
        else:
            print(f"다운로드 중: {file_path}")
            ydl_opts = {
                'format': 'bestaudio/best',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'mp3',
                    'preferredquality': '192',
                }],
                'outtmpl': f'{output_path}{title}.%(ext)s',
                'quiet': False,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([youtube_url])

    # 재생목록 처리
    else:
        print(f"재생목록에서 {len(info['entries'])}개의 동영상을 찾았습니다.")
        for entry in info['entries']:
            video_url = entry.get('url')
            title = entry.get('title', 'Unknown Title')  # 타이틀 자동 추출
            file_path = os.path.join(output_path, f"{title}.mp3")
            # 파일이 존재하는지 확인
            if os.path.exists(file_path):
                print(f"파일이 이미 존재합니다: {file_path}")
            else:
                print(f"다운로드 중: {file_path}")
                try:
                    ydl_opts = {
                        'format': 'bestaudio/best',
                        'postprocessors': [{
                            'key': 'FFmpegExtractAudio',
                            'preferredcodec': 'mp3',
                            'preferredquality': '192',
                        }],
                        'outtmpl': f'{output_path}{title}.%(ext)s',
                        'quiet': False,
                    }
                    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                        ydl.download([video_url])
                except Exception as e:
                    print(f"다운로드 실패: {title}, URL: {video_url}, 에러: {e}")

for row in video_parse_list.itertuples(index=False):
    path = row.path
    url = row.url
    noplaylist = row.noplaylist

    print(f"Path: {path}, URL: {url}, noplaylist: {noplaylist}")
    download_audio(url, f'./downloads/{path}/', noplaylist)

Path: 뱀서라이크, URL: https://www.youtube.com/playlist?list=PLO-mt5Iu5TeZF8xMHqtT_DhAPKmjF6i3x, noplaylist: False


재생목록에서 22개의 동영상을 찾았습니다.
파일이 이미 존재합니다: ./downloads/뱀서라이크/Unity Vampire Survivors Like Dev Course is coming?!🧟Undead Survivor.mp3
파일이 이미 존재합니다: ./downloads/뱀서라이크/Creating🧍2D Objects [Unity Vampire Survivors Like 01].mp3
파일이 이미 존재합니다: ./downloads/뱀서라이크/Implementing🚶Player Movement [Unity Vampire Survivors Like 02].mp3
파일이 이미 존재합니다: ./downloads/뱀서라이크/Applying The New Input System🎮 [Unity Vampire Survivors Like 02+].mp3
파일이 이미 존재합니다: ./downloads/뱀서라이크/Creating 2D Cell Animation🏃 [Unity Vampire Survivors Like 03].mp3
파일이 이미 존재합니다: ./downloads/뱀서라이크/Infinite🌍 Map Movement [Unity Vampire Survivors Like 04].mp3
파일이 이미 존재합니다: ./downloads/뱀서라이크/Creating Monster🧟 [Unity Vampire Survivors Like 05].mp3
파일이 이미 존재합니다: ./downloads/뱀서라이크/Summon by object pooling🏊 [Unity Vampire Survivors Like 06].mp3
파일이 이미 존재합니다: ./downloads/뱀서라이크/Apply Summon Level⏳ [Unity Vampire Survivors Like 06+].mp3
파일이 이미 존재합니다: ./downloads/뱀서라이크/Rotating 🪓 Melee Weapon [Unity Vampire Survivors Like 07].mp3
파일이 이미 존재합니다: ./downl

재생목록에서 12개의 동영상을 찾았습니다.
다운로드 중: ./downloads/비행기슈팅/2D 종스크롤 슈팅 -  플레이어 이동 구현하기 [유니티 기초 강좌 B27 + 에셋 다운로드].mp3
[youtube] Extracting URL: https://www.youtube.com/watch?v=ETYzjbnLixY
[youtube] ETYzjbnLixY: Downloading webpage
[youtube] ETYzjbnLixY: Downloading tv player API JSON
[youtube] ETYzjbnLixY: Downloading ios player API JSON
[youtube] ETYzjbnLixY: Downloading m3u8 information
[info] ETYzjbnLixY: Downloading 1 format(s): 251
[download] Destination: ./downloads/비행기슈팅/2D 종스크롤 슈팅 -  플레이어 이동 구현하기 [유니티 기초 강좌 B27 + 에셋 다운로드].webm
[download] 100% of   36.51MiB in 00:00:03 at 10.58MiB/s    
[ExtractAudio] Destination: ./downloads/비행기슈팅/2D 종스크롤 슈팅 -  플레이어 이동 구현하기 [유니티 기초 강좌 B27 + 에셋 다운로드].mp3
Deleting original file ./downloads/비행기슈팅/2D 종스크롤 슈팅 -  플레이어 이동 구현하기 [유니티 기초 강좌 B27 + 에셋 다운로드].webm (pass -k to keep)
다운로드 중: ./downloads/비행기슈팅/2D 종스크롤 슈팅 -  총알발사 구현하기 [유니티 기초 강좌 B28].mp3
[youtube] Extracting URL: https://www.youtube.com/watch?v=JUG0GnsJHQw
[youtube] JUG0GnsJHQw: Downloading webpage
[youtub

재생목록에서 8개의 동영상을 찾았습니다.
다운로드 중: ./downloads/탑다운RPG/탑다운 2D RPG - 도트 타일맵으로 쉽게 준비하기 [유니티 기초 강좌 B20 + 에셋 다운로드].mp3
[youtube] Extracting URL: https://www.youtube.com/watch?v=JY-KFx3OsJo
[youtube] JY-KFx3OsJo: Downloading webpage
[youtube] JY-KFx3OsJo: Downloading tv player API JSON
[youtube] JY-KFx3OsJo: Downloading ios player API JSON
[youtube] JY-KFx3OsJo: Downloading m3u8 information
[info] JY-KFx3OsJo: Downloading 1 format(s): 251
[download] Destination: ./downloads/탑다운RPG/탑다운 2D RPG - 도트 타일맵으로 쉽게 준비하기 [유니티 기초 강좌 B20 + 에셋 다운로드].webm
[download] 100% of   14.94MiB in 00:00:01 at 12.09MiB/s    
[ExtractAudio] Destination: ./downloads/탑다운RPG/탑다운 2D RPG - 도트 타일맵으로 쉽게 준비하기 [유니티 기초 강좌 B20 + 에셋 다운로드].mp3
Deleting original file ./downloads/탑다운RPG/탑다운 2D RPG - 도트 타일맵으로 쉽게 준비하기 [유니티 기초 강좌 B20 + 에셋 다운로드].webm (pass -k to keep)
다운로드 중: ./downloads/탑다운RPG/탑다운 2D RPG - 쯔꾸르식 액션 구현하기 [유니티 기초 강좌 B21].mp3
[youtube] Extracting URL: https://www.youtube.com/watch?v=bZVa6C6vRBQ
[youtube] bZVa6C6vRBQ: Downloadi

KeyboardInterrupt: 